## Mehrshad Saadatinia - 97243039

In [1]:
import pandas as pd
import sqlite3 as sql
import math

In [10]:
try:
    conn2 = sql.connect('population_data.db')
    print(conn2)
except sql.Error as error:
    print('connection error occurred')

In [11]:
cursor2 = conn2.cursor() #population

In [13]:
cursor2.execute("SELECT name FROM sqlite_master WHERE type='table';")
cursor2.fetchall()

[('sqlite_sequence',), ('facts',)]

In [22]:
pd.read_sql("SELECT * FROM facts;", conn2)

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
0,1,af,Afghanistan,652230.0,652230.0,0.0,3.256434e+07,2.32,38.57,13.89,1.51
1,2,al,Albania,28748.0,27398.0,1350.0,3.029278e+06,0.30,12.92,6.58,3.30
2,3,ag,Algeria,2381741.0,2381741.0,0.0,3.954217e+07,1.84,23.67,4.31,0.92
3,4,an,Andorra,468.0,468.0,0.0,8.558000e+04,0.12,8.13,6.96,0.00
4,5,ao,Angola,1246700.0,1246700.0,0.0,1.962535e+07,2.78,38.78,11.49,0.46
...,...,...,...,...,...,...,...,...,...,...,...
256,257,zh,Atlantic Ocean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
257,258,xo,Indian Ocean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
258,259,zn,Pacific Ocean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
259,260,oo,Southern Ocean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Question 2:  population analysis

In [62]:
#one 
max_pop = pd.read_sql("SELECT name, MAX(population) FROM facts WHERE name<>'World';", conn2)
min_pop = pd.read_sql("SELECT name, MIN(population) population FROM facts  WHERE name<>'Antarctica';", conn2)
max_pop_list = max_pop.values.tolist()[0]
min_pop_list = min_pop.values.tolist()[0]
print('one)\n')
print('max population belong to', max_pop_list[0], 'with', max_pop_list[1])
print('min population belong to', min_pop_list[0], 'with', min_pop_list[1])
print('--------------------------------------------------------------\n')

#two
max_pop_growth = pd.read_sql("SELECT name, MAX(population_growth) FROM facts;", conn2)
max_growth_list = max_pop_growth.values.tolist()[0]
print('two)\n')
print('max population growth belong to', max_growth_list[0], 'with', max_growth_list[1])
print('--------------------------------------------------------------\n')

#three
max_density = pd.read_sql("SELECT name, MAX((population / area)) FROM facts;", conn2)
min_density = pd.read_sql("SELECT name, MIN(CAST(population AS float) / CAST(area AS float)) FROM facts;", conn2)
max_density_list = max_density.values.tolist()[0]
min_density_list = min_density.values.tolist()[0]
print('three)\n')
print('max population density belong to', max_density_list[0], 'with', max_density_list[1])
print('min population density belong to', min_density_list[0], 'with', min_density_list[1])
print('--------------------------------------------------------------\n')

#four
top_ten_migr = pd.read_sql("SELECT name, migration_rate FROM facts ORDER BY migration_rate DESC LIMIT 10;", conn2)
print('four)\n')
print('top 10 most migration rate belongs to', [tuple(t) for t in top_ten_migr.values.tolist()])
print('--------------------------------------------------------------\n')

#five
top_five_birth_to_death = pd.read_sql("SELECT NAME,(birth_rate/death_rate) FROM facts ORDER BY (birth_rate/death_rate) DESC LIMIT 5;", conn2)
print('five)\n')
print('top 5 most birth_death rate belongs to', [tuple(t) for t in top_five_birth_to_death.values.tolist()])


one)

max population belong to China with 1367485388
min population belong to Pitcairn Islands with 48
--------------------------------------------------------------

two)

max population growth belong to South Sudan with 4.02
--------------------------------------------------------------

three)

max population density belong to Macau with 21168
min population density belong to Greenland with 0.026653143042335344
--------------------------------------------------------------

four)

top 10 most migration rate belongs to [('Qatar', 22.39), ('American Samoa', 21.13), ('Micronesia, Federated States of', 20.93), ('Syria', 19.79), ('Tonga', 17.84), ('British Virgin Islands', 17.28), ('Luxembourg', 17.16), ('Cayman Islands', 14.4), ('Singapore', 14.05), ('Nauru', 13.63)]
--------------------------------------------------------------

five)

top 5 most birth_death rate belongs to [('Gaza Strip', 10.233552631578947), ('Kuwait', 9.133027522935778), ('Iraq', 8.342175066312997), ('United Arab Em

In [64]:
def sqlite_power(x,n):
    return (x)**n

#add function to sqlite api
conn2.create_function("POWER", 2, sqlite_power)

In [67]:
numer_df = pd.read_sql("""SELECT 
 (COUNT(*)*SUM(death_rate*population_growth)) - (SUM(death_rate)*SUM(population_growth))
 AS numerator FROM facts WHERE (death_rate IS NOT NULL) AND (population_growth IS NOT NULL);""", conn2)
  
denom_df = pd.read_sql("""SELECT ABS( ( COUNT(*)*SUM(POWER(death_rate, 2)) ) - POWER(SUM(death_rate), 2) ) *
 ABS( (COUNT(*)*SUM(POWER(population_growth, 2))) - POWER(SUM(population_growth), 2) )
 AS denominator FROM facts WHERE (death_rate IS NOT NULL) AND (population_growth IS NOT NULL);""", conn2)  
    
denom = denom_df['denominator']
numer = numer_df['numerator']
print('six) \n ')
numer/math.sqrt(denom)

six) 
 


0   -0.101521
Name: numerator, dtype: float64

##### result
result is **-0.101** which is a negative correlation (makes sense because more death means less population growth)
and the magnitude of correlation is 0.101 which is  considered a **weak correlation**

**so death_rate and population growth are weakly correlated**